In [106]:
import pandas as pd
from sklearn import svm




In [1]:
test_data = pd.read_csv("test.csv")
train_data = pd.read_csv("train.csv")
feat = train_data.columns

# Data manipulation
mapping = {'S':1, 'C':2, 'Q':3}
#test_data['Embarked'] = [mapping[i] for i in test_data['Embarked']]
#train_data['Embarked'] = [mapping[i] for i in train_data['Embarked']]
test_data.replace(to_replace=['S', 'C', 'Q'], value=[1,2,3], inplace=True)
train_data.replace(to_replace=['S', 'C', 'Q'], value=[1,2,3], inplace=True)

test_data.replace(to_replace=['male', 'female'], value=[1,2], inplace=True)
train_data.replace(to_replace=['male', 'female'], value=[1,2], inplace=True)

# Dropping NaNs
#test_data.Age = test_data.Age.fillna(0, inplace= True)
#train_data.Age = train_data.Age.fillna(0, inplace = True)

test_data = test_data.fillna(0)
train_data = train_data.fillna(0)

used_feats = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
# 'Ticket', 'Cabin'
train_x = train_data[used_feats]
test_x  = test_data[used_feats]
train_y = train_data['Survived']
test_id = test_data['PassengerId']
 

NameError: name 'pd' is not defined

In [112]:
#print(train_x)
#print(train_y)

# Training a SVM with RBF
model = svm.SVC(kernel='rbf', gamma=1, C=1000)
model.fit(train_x, train_y)
pred = model.predict(test_x)

# Cross validation

df_pred = pd.DataFrame(test_id, columns = ['PassengerId'])
df_pred['Survived'] = pred
#print(df_pred)

# Saving as csv
preds = df_pred.to_csv('dbjrelind2_pred.csv', index = False)


Notes no self:

- Do metric for test preds
- Find what params are best
- Cross validation?